In [48]:
# This note book use nearest neighbor method(The most closest cell line) to predict CERES score of each gene. 

# Author: Yiyun

import pandas as pd
from os.path import join
from scipy.spatial import distance
from sklearn.metrics import r2_score

In [53]:
### Read celligner and 19q3 file
celligner_dir = '../data/DepMap/celligner'
q3_dir = '../data/DepMap/19Q3'

df_celligner = pd.read_csv(join(celligner_dir,'celligner_alignment.csv'),index_col = 0)
df_ref19q3 = pd.read_csv(join(q3_dir,'Achilles_gene_effect.csv'), index_col = 0)

# Check if the UMAP data is consistant as on the Celligner website
# df_cells.plot.scatter(x='UMAP_1',
#                       y='UMAP_2')

In [54]:
### Not all 19q3 cell lines are in celligner, we find nearest neighbors in 19q3 dataset
### Therefore the closest cell line is not necessarily the one in the Celligner

# Get the cell line data in celligner
df_celligner = df_celligner[df_celligner['sampleID'].str.startswith('ACH-')][['sampleID','UMAP_1','UMAP_2']].transpose()
new_header = df_celligner.iloc[0] 
df_celligner = df_celligner[1:] 
df_celligner.columns = new_header 

# Select 19q3 cell line in Celligner cell lines, 3 cell lines are not found, process as missing data
cells_19q3 = df_ref19q3.index
cells_celligner = df_celligner.columns

# Check if there's cell lines in 19q3 not in celligner
cells_NF = [q3_cell for q3_cell in cells_19q3 if q3_cell not in cells_celligner]

# drop those cell lines from 19q3 list
cells_F = [q3_cell for q3_cell in cells_19q3 if q3_cell not in cells_NF]

# Select q3 cell line in celligner
df_celligner = df_celligner[cells_F]

In [56]:
### Calculate euclidean distance for each samples
df_close_cells = pd.DataFrame(index = ['closest_cell_line','distance'])

for cells in df_celligner.columns:
    rest_cells = df_celligner.columns.drop(cells)
    
    dist_0 = float(10000) # initiate a biggest value to find the smallest value
    close_cell = 'Random'
    for rest_cell in rest_cells:
        dist = distance.euclidean(df_celligner[cells],df_celligner[rest_cell])
        
        # Update the smallest distance
        if dist <= dist_0:
            dist_0=dist
            close_cell = rest_cell
            
    df_close_cells[cells] = [close_cell,dist_0]

In [57]:
df_close_cells
# df_close_cells.to_csv('nearest_neighbor_cell_dist.csv')

,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,ACH-000015,ACH-000017,...,ACH-001715,ACH-001735,ACH-001736,ACH-001737,ACH-001740,ACH-001745,ACH-001750,ACH-001765,ACH-001814,ACH-001838
closest_cell_line,ACH-000005,ACH-000004,ACH-000820,ACH-000296,ACH-000234,ACH-000528,ACH-000293,ACH-000322,ACH-000221,ACH-000248,...,ACH-000673,ACH-001736,ACH-000782,ACH-000953,ACH-001765,ACH-001765,ACH-000133,ACH-001745,ACH-000082,ACH-000042
distance,0.230702,0.230702,0.709248,0.395715,0.212912,0.347879,0.206121,0.0550587,0.128074,0.325979,...,0.0737915,0.369783,0.274713,0.705307,0.716216,0.144185,0.257814,0.144185,0.10245,0.0516301


In [ ]:
### Fill the prediction dataframe by the value of the closest cell line

# Create a prediction dataframe
index_cellline = df_ref19q3.index
column_genes = df_ref19q3.columns
df_prediction = pd.DataFrame(index = index_cellline, columns = column_genes)

# Update the CERES score with the score of closest cell line
for genes in column_genes:
    for cells in index_cellline:
        try:
            close_cellline = df_close_cells.loc[0,cells]
            pred_score = df_ref19q3.loc[close_cellline,genes]
            df_prediction.loc[cells,genes] = pred_score
        except KeyError:
            df_prediction.loc[cells,genes] = None

In [ ]:
# df_prediction
# df_prediction.to_csv('prediction_040621.csv')
# df_prediction = pd.read_csv('prediction_040621.csv', index_col = 0)

In [17]:
### Calculate R2 value for most diverse genes

### Read in data to get the most diverse gene list
dir_in_res = '../out/20.0216 feat/reg_rf_boruta'
dir_in_anlyz = join(dir_in_res, 'anlyz_filtered')
f_featsumm = join(dir_in_anlyz,'agg_summary_filtered.csv')
df_aggRes = pd.read_csv(f_featsumm) #aggregated feat summary
div_genes = df_aggRes['target']

In [32]:
### map the div_gene name to the name in prediction dataframe
list_div_genes = []
for gene_idx in df_prediction.columns:
    gene_name = gene_idx.split(' ')[0]
    if gene_name in div_genes.to_list():
        list_div_genes.append(gene_idx)

In [41]:
### Plot the correlation plot between actual and predicted, for every gene
df_corr_gene = pd.DataFrame(columns = ['gene','r2(score_test)'])

list_r2 = []
for gene in list_div_genes:
    df_score = pd.DataFrame(columns = ['actual','predicted'])
    df_score['actual'] = df_ref19q3.loc[:,gene]
    df_score['predicted'] = df_prediction.loc[:,gene]
    df_score = df_score.dropna()
    score_r2 = r2_score(df_score['actual'],df_score['predicted'])
    list_r2.append(score_r2)
    
df_corr_gene['gene'] = div_genes; df_corr_gene['r2(score_test)']

In [46]:
df_corr_gene

,gene,r2(score_test)
0,ACACA (31),-0.751065
1,ACLY (47),-0.521512
2,ACO2 (50),-0.593567
3,ACSL3 (2181),-0.518309
4,ACTR1A (10121),-0.814175
...,...,...
524,YTHDF2 (51441),-0.853425
525,ZEB2 (9839),0.172422
526,ZFP36L1 (677),-0.493024
527,ZMYND8 (23613),-0.626991


In [45]:
# Median of all r2 value
df_corr_gene['r2(score_test)'].median()

-0.7057783637697985